In [ ]:
#Optimización de colonias de hormigas para resolver el problema de TSP
import numpy as np
import random as rd

def lengthCal(antPath,distmat):         #Calcular distancia
    length =[]
    dis = 0
    for i in range(len(antPath)):
        for j in range(len(antPath[i]) - 1):
            dis += distmat[antPath[i][j]][antPath[i][j + 1]]
        dis += distmat[antPath[i][-1]][antPath[i][0]]
        length.append(dis)
        dis = 0
    return length

import math
vertices = np.array([[255,380],  # Cristo Redentor (BRAZIL)
                     [575,236],  # Taj Mahal (INDIA)
                     [412,179],  # Coliseo Romano (ITALIA)
                     [667,183],  # Gran Muralla China (CHINA)
                     [141,247],  # Chichén Itzá (MEXICO)
                     [173,344],  # Macchu Picchu (PERU)
                     [466,216]])


distmat = np.zeros((len(vertices),len(vertices)))
for i in range(len(vertices)):
    for j in range(len(vertices)):
        distmat[i][j] = math.sqrt(pow(vertices[i][0] - vertices[j][0],2) + pow(vertices[i][1] - vertices[j][1],2))


antNum = 7                   # Número de hormiga
alpha = 1                     # Factor de importancia de las feromonas
beta = 3                      # Factor de importancia de la función heurística
pheEvaRate = 0.3              # Tasa de evaporación de feromonas
cityNum = distmat.shape[0]
pheromone = np.ones((cityNum,cityNum))                   # Matriz de feromonas
heuristic = 1 / (np.eye(cityNum) + distmat) - np.eye(cityNum)       # Matriz de información heurística, tome 1 / dismat
iter,itermax = 1,100                       # Número de iteraciones

while iter < itermax:
    antPath = np.zeros((antNum, cityNum)).astype(int) - 1   # Camino de la hormiga
    firstCity = [i for i in range(7)]
    rd.shuffle(firstCity)          # Asigna aleatoriamente una ciudad de inicio para cada hormiga
    unvisted = []
    p = []
    pAccum = 0
    for i in range(len(antPath)):
        antPath[i][0] = firstCity[i]
    for i in range(len(antPath[0]) - 1):       # Actualiza gradualmente la próxima ciudad a la que irá cada hormiga
        for j in range(len(antPath)):
            for k in range(cityNum):
                if k not in antPath[j]:
                    unvisted.append(k)
            for m in unvisted:
                pAccum += pheromone[antPath[j][i]][m] ** alpha * heuristic[antPath[j][i]][m] ** beta
            for n in unvisted:
                p.append(pheromone[antPath[j][i]][n] ** alpha * heuristic[antPath[j][i]][n] ** beta / pAccum)
            roulette = np.array(p).cumsum()               #Generar ruleta
            r = rd.uniform(min(roulette), max(roulette))
            for x in range(len(roulette)):
                if roulette[x] >= r:                      #Utilice el método de la ruleta para elegir la próxima ciudad para ir
                    antPath[j][i + 1] = unvisted[x]
                    break
            unvisted = []
            p = []
            pAccum = 0
    pheromone = (1 - pheEvaRate) * pheromone            # Feromonas volátiles
    length = lengthCal(antPath,distmat)
    for i in range(len(antPath)):
        for j in range(len(antPath[i]) - 1):
            pheromone[antPath[i][j]][antPath[i][j + 1]] += 1 / length[i]     #Actualización de feromonas
        pheromone[antPath[i][-1]][antPath[i][0]] += 1 / length[i]
    iter += 1
print("La distancia más corta es:")
print(min(length))
print("El camino más corto es:")
print(antPath[length.index(min(length))])
